In [1]:
# Imports

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from datetime import datetime

import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='halima.schede', api_key='0BXIz4i3MnnYF4z7QhA0')

from helpers import *
from timeline_helpers import *


In [2]:
# Load data
tweets1 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_1.csv')
tweets2 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_2.csv')
tweets3 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_3.csv')
tweets4 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_4.csv')
tweets5 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_5.csv')
tweets6 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_6.csv')
tweets7 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_7.csv')
tweets8 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_8.csv')
tweets9 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_9.csv')



tweets = pd.concat([tweets1, tweets2, tweets3, tweets4, tweets5, tweets6, tweets7, tweets8, tweets9], axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)


In [3]:
tweets.describe()

,external_author_id,following,followers,updates,new_june_2018,retweet
count,2.973367e+06,2.973371e+06,2.973371e+06,2.973371e+06,2.973371e+06,2.973371e+06
mean,1.296128e+17,3.433524e+03,7.018913e+03,1.049756e+04,2.078735e-01,4.408955e-01
std,3.036341e+17,5.609881e+03,1.458463e+04,1.768729e+04,4.057859e-01,4.964945e-01
min,3.497640e+07,-1.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00
25%,1.930748e+09,3.270000e+02,3.200000e+02,1.787000e+03,0.000000e+00,0.000000e+00
50%,2.581835e+09,1.499000e+03,1.274000e+03,4.333000e+03,0.000000e+00,0.000000e+00
75%,3.254274e+09,4.730000e+03,1.060000e+04,1.234100e+04,0.000000e+00,1.000000e+00
max,9.812510e+17,7.621000e+04,2.512760e+05,1.661130e+05,1.000000e+00,1.000000e+00


In [4]:
tweets.drop(['new_june_2018','harvested_date','content','region','retweet']
            ,axis=1, inplace=True)

In [5]:
# check for NaN values
tweets.isnull().sum()

external_author_id          4
author                      0
language                    0
publish_date                0
following                   0
followers                   0
updates                     0
post_type             1662425
account_type              363
account_category            0
dtype: int64

In [6]:
# convert datetime * respect m/d/y format
tweets['publish_date'] = pd.to_datetime(tweets['publish_date'], format='%m/%d/%Y %H:%M')

### Language Distribution

In [7]:
# create dataframe with publish_date and language
language_time = tweets[['publish_date', 'language']]


In [8]:
data = trace_generator_language(language_time)
layout = do_layout('Date', 'Number of Tweets', 'Language as a Function of Time')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styling-names')

In [9]:
# Since the values of tweets are minimal before 2015 filter dataframe
tweets_processed = tweets[tweets.publish_date > '2015']

### Account Category

In [10]:
#create dataframe with publish date and account type
accounttype_time = tweets_processed[['publish_date', 'account_category']]

In [11]:
data = trace_generator_account(accounttype_time)
layout = do_layout('Date', 'Number of Tweets','Account Type of Tweets by Month')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styling-names2')

In [12]:
bubble_data = tweets_processed[['publish_date', 'followers','account_category']]

righttroll = count_followers('RightTroll', bubble_data)
lefttroll = count_followers('LeftTroll', bubble_data)
newsfeed = count_followers('NewsFeed', bubble_data)
nonenglish = count_followers('NonEnglish', bubble_data)

In [13]:
trace0 = go.Scatter(
    x=list(pd.Series(righttroll.index.strftime('%Y-%m-%d %H-%M-%S')).apply(lambda x: x[0:7])),
    y=righttroll.tweet_category_num.values,
    mode='markers',
    name='Right Troll',
    marker=dict(
        size=righttroll.followers.values * 0.00001,
        sizemode='area',
        sizeref=2,
        sizemin=4))

trace1 = go.Scatter(
    x=list(pd.Series(lefttroll.index.strftime('%Y-%m-%d %H-%M-%S')).apply(lambda x: x[0:7])),
    y=lefttroll.tweet_category_num.values,
    mode='markers',
    name='Left Troll',
    marker=dict(
        size=lefttroll.followers.values * 0.00001,
        sizemode='area',
        sizeref=2,
        sizemin=4))

In [14]:
data = [trace0, trace1]
layout = do_layout('Date','Number of Tweets in Category',
                   'Social Power of Tweet Category by Month')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='styling-names2')